# Author : Vitor Henrique Mendes

In [1]:
// Importing Maven Dependencies at Runtime Using Almond Kernel
import $ivy.`io.delta:delta-core_2.11:0.4.0`
import $ivy.`org.apache.spark:spark-core_2.11:2.4.4`
import $ivy.`org.apache.spark:spark-sql_2.11:2.4.4`
import $ivy.`org.apache.spark:spark-catalyst_2.11:2.4.4`
import $ivy.`log4j:log4j:1.2.17`

import $ivy.$                               

import $ivy.$                                       

import $ivy.$                                      

import $ivy.$                                           

import $ivy.$                   

In [2]:
import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.apache.spark.storage.{StorageLevel => Storage}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

import io.delta.tables.DeltaTable

import org.apache.log4j.Logger
import org.apache.log4j.Level

Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark.SparkConf

import org.apache.spark.sql.SparkSession

import org.apache.spark.storage.{StorageLevel => Storage}

import org.apache.spark.sql.functions._

import org.apache.spark.sql.types._


import io.delta.tables.DeltaTable


import org.apache.log4j.Logger

import org.apache.log4j.Level



In [3]:
val appConf = new SparkConf().setAll(Array(
    ("spark.driver.memory","1g"),
    ("spark.executor.memory","2g"),
    ("spark.executor.cores","4"),
    ("spark.executor.instances","2")
))
// Running on Standalone Mode
val spark = SparkSession
                .builder
                .appName("Semantix")
                .master("spark://vitor-VirtualBox:7077")
                .config(appConf)
                .getOrCreate

import spark.implicits._

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


appConf: SparkConf = org.apache.spark.SparkConf@42c630dd
spark: SparkSession = org.apache.spark.sql.SparkSession@2fe601d9
import spark.implicits._

In [4]:
val rawDF = spark.read.option("escape","\"").option("multiLine","true").text("data/*.gz")
rawDF.show(1,false)

+--------------------------------------------------------------------------------------+
|value                                                                                 |
+--------------------------------------------------------------------------------------+
|199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245|
+--------------------------------------------------------------------------------------+
only showing top 1 row



rawDF: org.apache.spark.sql.package.DataFrame = [value: string]

In [17]:
// Parsing and Cleaning
val hostRegex : String = "([0-9]{1,3})\\.([0-9]{1,3})\\.([0-9]{1,3})\\.([0-9]{1,3})"
val df = rawDF
            .select(split($"value"," ") as "items")
            .select(
                $"items".getItem(0) as "origin",
                when(regexp_extract($"items".getItem(0),hostRegex,1) =!= "", "host").otherwise("url") as "origin_type", 
                translate($"items".getItem(3),"[","") as "timestamp",
                translate(trim($"items".getItem(5)),"\"","") as "request_type",
                $"items".getItem(6) as "request_destine",
                $"items".getItem(8).cast(IntegerType) as "request_code",
                $"items".getItem(9).cast(FloatType) as "total_bytes"
            )
            .persist(Storage.MEMORY_ONLY) // Same as df.cache()
df.count() // An action to real cache data into Memory

hostRegex: String = "([0-9]{1,3})\\.([0-9]{1,3})\\.([0-9]{1,3})\\.([0-9]{1,3})"
df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [origin: string, origin_type: string ... 5 more fields]
res16_2: Long = 3461613L

In [18]:
df.show(10,false)

+--------------------+-----------+--------------------+------------+-----------------------------------------------+------------+-----------+
|origin              |origin_type|timestamp           |request_type|request_destine                                |request_code|total_bytes|
+--------------------+-----------+--------------------+------------+-----------------------------------------------+------------+-----------+
|199.72.81.55        |host       |01/Jul/1995:00:00:01|GET         |/history/apollo/                               |200         |6245.0     |
|unicomp6.unicomp.net|url        |01/Jul/1995:00:00:06|GET         |/shuttle/countdown/                            |200         |3985.0     |
|199.120.110.21      |host       |01/Jul/1995:00:00:09|GET         |/shuttle/missions/sts-73/mission-sts-73.html   |200         |4085.0     |
|burger.letters.com  |url        |01/Jul/1995:00:00:11|GET         |/shuttle/countdown/liftoff.html                |304         |0.0        |
|199.1

# Question 1

In [8]:
val distintHosts : Long = df.select($"host").distinct().count()
println(s"Number of Distinct Hosts are : ${distintHosts}")

Number of Distinct Hosts are : 137979


distintHosts: Long = 137979L

# Question 2

In [9]:
val notFoundRequests : Long = df.where($"request_code" === 404).count()
println(s"Number of Requests with code 404 are : ${notFoundRequests}")

Number of Requests with code 404 are : 20686


notFoundRequests: Long = 20686L

# Question 3

In [19]:
df
 .where($"origin_type" === "url" && $"request_code" === 404)
 .groupBy("origin")
 .agg(count(lit(1)) as "qtde")
 .orderBy($"qtde".desc)
 .limit(5)
 .show(false)

+---------------------------+----+
|origin                     |qtde|
+---------------------------+----+
|hoohoo.ncsa.uiuc.edu       |251 |
|piweba3y.prodigy.com       |157 |
|jbiagioni.npt.nuwc.navy.mil|132 |
|piweba1y.prodigy.com       |114 |
|www-d4.proxy.aol.com       |91  |
+---------------------------+----+



# Question 4

In [80]:
val parsedDates = df.select($"request_code",regexp_extract($"timestamp","\\d{2}/\\w{1,6}/\\d{2,4}",0) as "timestamp")
                    .select(
                         $"request_code",
                          when($"timestamp".isNotNull && lower($"timestamp").contains("jul") , translate($"timestamp","Jul","07"))
                          .when($"timestamp".isNotNull && lower($"timestamp").contains("aug"), translate($"timestamp","Aug","08"))
                          .as("timestamp")
                      )
                    .select($"request_code",to_date($"timestamp","dd/MM/yyyy") as "timestamp")

parsedDates.show(5)

+------------+----------+
|request_code| timestamp|
+------------+----------+
|         200|1995-07-01|
|         200|1995-07-01|
|         200|1995-07-01|
|         304|1995-07-01|
|         200|1995-07-01|
+------------+----------+
only showing top 5 rows



parsedDates: org.apache.spark.sql.package.DataFrame = [request_code: int, timestamp: date]

In [85]:
parsedDates
    .where("request_code = 404")
    .groupBy("timestamp")
    .agg(count(lit(1)) as "qtde")
    .orderBy($"timestamp".asc)
    .show()

+----------+----+
| timestamp|qtde|
+----------+----+
|1995-07-01| 314|
|1995-07-02| 289|
|1995-07-03| 473|
|1995-07-04| 355|
|1995-07-05| 491|
|1995-07-06| 630|
|1995-07-07| 564|
|1995-07-08| 299|
|1995-07-09| 341|
|1995-07-10| 390|
|1995-07-11| 469|
|1995-07-12| 459|
|1995-07-13| 524|
|1995-07-14| 408|
|1995-07-15| 252|
|1995-07-16| 256|
|1995-07-17| 403|
|1995-07-18| 463|
|1995-07-19| 636|
|1995-07-20| 427|
+----------+----+
only showing top 20 rows



# Question 5

In [22]:
df.agg(
    sum($"total_bytes") as "total_in_bytes"
).show()

+---------------+
| total_in_bytes|
+---------------+
|6.5143741103E10|
+---------------+

